ЗАГРУЗИМ ПОДГОТОВЛЕННЫЕ ДАННЫЕ 50 СТАТЕЙ ИЗ WIKIPEDIA В SOLR И ВЫПОЛНИМ НЕСКОЛЬКО ЗАПРОСОВ ПО ПОИСКУ НАШИХ ДОКУМЕНТОВ

Данильченко Вадим

In [4]:
!pip3 install pysolr

In [5]:
# импортируем библиотеки
import pysolr
import json
from tqdm import tqdm

In [9]:
# Создаем клиент solr, указывая адрес, порт и название core (можно посмотреть в Web интерфейсе)
solr = pysolr.Solr('http://0.0.0.0:8983/solr/gettingstarted/', always_commit=True)

In [10]:
# Проверим доступность solr
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":98,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"-0"}},\n  "status":"OK"}\n'

In [11]:
# посмотрим на наличие нужных файлов в корневой директории
!ls

'extract data from wikipedia.ipynb'  'solr requests.ipynb'
 myvenv				      wikipages_50.json


In [16]:
# загрузим подготовленные данные статей википедии
with open('./wikipages_50.json', "r") as f:
    pages = json.load(f)
    
pages['0']

{'content': 'Rhynchosia minima is a species of flowering plant in the legume family known by the common names least snout-bean, burn-mouth-vine, and jumby-bean. It can be found on every continent. It is naturalized in Hawaii.This perennial herb has twining or trailing stems which can reach 1.2 meters in length. The leaves are made up of three leaflets measuring up to 3 to 3.5 centimeters long. The inflorescence is a raceme of up to 15 flowers. The flowers are yellow with purple or brown veining and measure up to 8 millimeters long. The fruit is one or two centimeters long.This common plant is used as animal forage.\n\n\n== References ==\n\n\n== External links ==\nRhynchosia minima in West African plants – A Photo Guide.',
 'meta': {'title': 'Rhynchosia minima',
  'categories': ['All stub articles',
   "Articles with 'species' microformats",
   'Articles with short description',
   'Flora of Australia',
   'Phaseoleae stubs',
   'Rhynchosia',
   'Short description is different from Wiki

In [21]:
# выполним добавление только метаданных
for k,v in tqdm(pages.items()):
    solr.add([
    {
        "id": "doc_" + k,
        "title": v['meta']['title'],
        "categories": v['meta']['categories']
        }])

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:06<00:00,  7.40it/s]


In [22]:
# удалим все записи по ключу документа
for k,v in tqdm(pages.items()):
    solr.delete(id="doc_"+k)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 15.19it/s]


In [25]:
# выполним проверку удаленного документа, осуществив поиск по заголовку
for res in solr.search('title:Rhynchosia minima'):
    print(res['title'])

In [27]:
# выполним добавление всех данных из каждой страницы
for k,v in tqdm(pages.items()):
    solr.add([
    {
        "id": "doc_" + k,
        "title": v['meta']['title'],
        "categories": v['meta']['categories'],
        "text": v["content"]
        }])

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:06<00:00,  7.32it/s]


In [28]:
# выполним проверку добавленного документа, осуществив поиск по заголовку
for res in solr.search('title:Rhynchosia minima'):
    print(res['title'])

['Rhynchosia minima']


In [33]:
# выполним запрос, указав заголовок документа
print("page 31 title original: ", pages["31"]["meta"]["title"])
for res in solr.search('title:'+pages["31"]["meta"]["title"]):
    print("res title: ", res['title'])
    print("res text: \n", res['text'])

page 31 title original:  Cerro Colorado Mountains
res title:  ['Cerro Colorado Mountains']
res text: 
 ['The Cerro Colorado Mountains are a low mountain range in southern Pima County, Arizona, USA. The highest point of the range is (5,319 feet (1,621 m)). The range consists of a NNW–SSE trending ridge with several shorter ridges extending off the main ridge to the ENE. The higher portions of the range cover an area of about 21 km2 (8.1 sq mi), with dimensions of 6.8 km (4.2 mi) by 4.0 km (2.5 mi).The foothills of the Sierrita Mountains lie about 7 mi (11 km) to the north with the Batamote Hills and Penitas Hills between to the north and northwest respectively. The Las Guijas Mountains lie 12 km (7.5 mi) to the southwest. The Sierra Colorado silver mining district lies in the hills between the Sierra Colorados and the Las Guijas ranges. Saucito Mountain at the north end of the Tumacacori Mountains of Santa Cruz County is about 11 km (6.8 mi) southeast. Arivaca Junction in the Santa Cruz

In [64]:
# выполним поиск по фрагменту текста
print("page 42 text fragment original: ", pages["42"]["content"][22:30])
print("page 42 title original: ", pages["42"]["meta"]["title"])
for res in solr.search("text:" + pages["42"]["content"][22:30]):
    print("res title: ", res['title'])
    print("res text: \n", res['text'])
    print("res categories: \n", res['categories'])

page 42 text fragment original:  Response
page 42 title original:  FBI Critical Incident Response Group
res title:  ['FBI Critical Incident Response Group']
res text: 
 ['The Critical Incident Response Group (CIRG) is a division of the Criminal, Cyber, Response, and Services Branch of the United States Federal Bureau of Investigation. CIRG enables the FBI to rapidly respond to, and effectively manage, special crisis incidents in the United States.\n\n\n== History ==\nIn response to public outcry over the standoffs at Ruby Ridge, Idaho, and of the Branch Davidians in the Waco Siege, the FBI formed the CIRG in 1994 to deal more efficiently with crisis situations. The CIRG is designated to formulate strategies, manage hostage or siege situations, and, if humanly possible, resolve them "without loss of life," as FBI Director Louis Freeh, who assumed the post four-and-a-half months after the Waco fire, pledged in a 1995 Senate hearing.\nCIRG was intended to integrate tactical and investigat